### Imports

In [ ]:
import numpy as np
import pybamm

from aepybamm import get_params

### Define source parameters

We use the About:Energy Gen1 demo cell (fictitious cell for technology demonstration).

In [ ]:
fp_bpx = "params/AE_gen1_BPX.json"

### Call `get_params()` with a defined degradation state

Compare the vanilla approach in `base_example.ipynb`. Here, the `SOC_definition` input to `get_params()` defines the initial SOC of 90% to be with reference to the capacity in the thermodynamic voltage window from 3 V to 4.1 V, rather than the full window between the operating cut-off voltages (2.5 V to 4.2 V). This means that SOC 90% occurs at lower OCV than in the reference example.

*Note: The warning returned from the BPX validator is due to rounding error in the stoichiometry specification of the parameter set.*

In [ ]:
SOC_definition = {
    "method": "linear_endpoints",
    "data": np.array(
        [
            [0, 3],
            [1, 4.1],
        ]
    ),
}

parameter_values, model = get_params(
    fp_bpx,
    SOC_init=0.9,
    SOC_definition=SOC_definition,
)

experiment = pybamm.Experiment(
    [
        "Discharge at 1C until 3 V",
        "Rest for 30 minutes",
        "Charge at 0.2C until 3.6 V",
    ]
)

sim = pybamm.Simulation(
    model,
    parameter_values=parameter_values,
    experiment=experiment,
)
sol = sim.solve(calc_esoh=False)
sol.plot();